You should never clean data manually, you should use code to automate cleaning tasks and minimize repetition.

Programmatic data cleaning has 3 steps:
- Define
- Code
- Test

The very first thing to do before any cleaning occurs is to make a copy of each piece of data. All of the cleaning operations will be conducted on this copy so you can still view the original dirty and/or messy dataset later. Copying DataFrames in pandas is done using the `copy` method. If the original DataFrame was called df, the soon-to-be clean copy of the dataset could be named df_clean.

In [140]:
import pandas as pd
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')

In [141]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
adverse_reactions_clean = adverse_reactions.copy()

### Quality

#### patients table

- zip code is a float not a string
- zip code has four gigits sometimes
- Tim Neudorf height is 27 inches instead of 72 inches
- State has some data in full name, some in abbreviations
- Dsvid Gustafsson
- Missing demographic information (address to contract)
- Erroneous datetype (assigned sex(should be categrocal datatype), state, zip_code, birthdate)
- multiple phone number formate
- Default John Doe data
- Multiple records for Jackbsen, Gersten, Taylor
- kgs instrad of lbs for Zaitseva weight

#### treatment table

- missing HbA1c changes
- the letter 'u' in starting and ending doese for auralin and novovrda
- lowercase given names and surnames
- missing records (280 instead of 350)
- Erroneous datetype (auralin and novodra columns)
- Inaccurate HbA1c_changes (4s mistaken as 9s)
- Nulls represented as dashes (-) in auralin and novodra columns

#### adverse_reactions table

- lowercase given names and surnames

### Tidiness

- contact column in `patients` table should be split into phone number and email address
- three variables in two columns in treatments table(treatment, start does and end does)

## Clean

### Missing Data

- missing HbA1c changes
- missing records (280 instead of 350)
- Missing demographic information (address to contract): _there is no idea to deal with this information missing, for we do not have any contact information, maybe we could skim the phone book or the internet for their contact information, but it's more efficient to hold on until they come back to our office._

#### Define

#### [Knowledge: Imputation (statistics)](https://en.wikipedia.org/wiki/Imputation_(statistics))

In statistics, imputation is the process of replacing missing data with substituted values. When substituting for a data point, it is known as "unit imputation"; when substituting for a component of a data point, it is known as "item imputation". There are three main problems that missing data causes: missing data can introduce a substantial amount of bias, make the handling and analysis of the data more arduous, and create reductions in efficiency.[1] Because missing data can create problems for analyzing data, imputation is seen as a way to avoid pitfalls involved with listwise deletion of cases that have missing values. That is to say, when one or more values are missing for a case, most statistical packages default to discarding any case that has a missing value, which may introduce bias or affect the representativeness of the results. Imputation preserves all cases by replacing missing data with an estimated value based on other available information. Once all missing values have been imputed, the data set can then be analysed using standard techniques for complete data.[2] Imputation theory is constantly developing and thus requires consistent attention to new information regarding the subject. There have been many theories embraced by scientists to account for missing data but the majority of them introduce large amounts of bias. A few of the well known attempts to deal with missing data include: hot deck and cold deck imputation; listwise and pairwise deletion; mean imputation; regression imputation; last observation carried forward; stochastic imputation; and multiple imputation.

##### 1. Listwise (complete case) deletion
By far, the most common means of dealing with missing data is listwise deletion (also known as complete case), which is when all cases with a missing value are deleted. If the data are missing completely at random, then listwise deletion does not add any bias, but it does decrease the power of the analysis by decreasing the effective sample size. For example, if 1000 cases are collected but 80 have missing values, the effective sample size after listwise deletion is 920. If the cases are not missing completely at random, then listwise deletion will introduce bias because the sub-sample of cases represented by the missing data are not representative of the original sample (and if the original sample was itself a representative sample of a population, the complete cases are not representative of that population either). While listwise deletion is unbiased when the missing data is missing completely at random, this is rarely the case in actuality.[3]

Pairwise deletion (or "available case analysis") involves deleting a case when it is missing a variable required for a particular analysis, but including that case in analyses for which all required variables are present. When pairwise deletion is used, the total N for analysis will not be consistent across parameter estimations. Because of the incomplete N values at some points in time, while still maintaining complete case comparison for other parameters, pairwise deletion can introduce impossible mathematical situations such as correlations that are over 100%.[4]

The one advantage complete case has over other methods is that it is straightforward and easy to implement. This is a large reason why complete case is the most popular method of handling missing data in spite of the many disadvantages it has.

##### 2. Single imputation
###### a) Hot-deck
A once-common method of imputation was hot-deck imputation where a missing value was imputed from a randomly selected similar record. The term "hot deck" dates back to the storage of data on punched cards, and indicates that the information donors come from the same dataset as the recipients. The stack of cards was "hot" because it was currently being processed.     
One form of hot-deck imputation is called "last observation carried forward" (or LOCF for short), which involves sorting a dataset according to any of a number of variables, thus creating an ordered dataset. The technique then finds the first missing value and uses the cell value immediately prior to the data that are missing to impute the missing value. The process is repeated for the next cell with a missing value until all missing values have been imputed. In the common scenario in which the cases are repeated measurements of a variable for a person or other entity, this represents the belief that if a measurement is missing, the best guess is that it hasn't changed from the last time it was measured. This method is known to increase risk of increasing bias and potentially false conclusions. For this reason LOCF is not recommended for use.[5]

###### b) Cold-deck
Cold-deck imputation, by contrast, selects donors from another dataset. Due to advances in computer power, more sophisticated methods of imputation have generally superseded the original random and sorted hot deck imputation techniques.

###### c) Mean substitution
Another imputation technique involves replacing any missing value with the mean of that variable for all other cases, which has the benefit of not changing the sample mean for that variable. However, mean imputation attenuates any correlations involving the variable(s) that are imputed. This is because, in cases with imputation, there is guaranteed to be no relationship between the imputed variable and any other measured variables. Thus, mean imputation has some attractive properties for univariate analysis but becomes problematic for multivariate analysis.

###### d) Regression
Regression imputation has the opposite problem of mean imputation. A regression model is estimated to predict observed values of a variable based on other variables, and that model is then used to impute values in cases where the value of that variable is missing. In other words, available information for complete and incomplete cases is used to predict the value of a specific variable. Fitted values from the regression model are then used to impute the missing values. The problem is that the imputed data do not have an error term included in their estimation, thus the estimates fit perfectly along the regression line without any residual variance. This causes relationships to be over identified and suggest greater precision in the imputed values than is warranted. The regression model predicts the most likely value of missing data but does not supply uncertainty about that value.    
Stochastic regression was a fairly successful attempt to correct the lack of an error term in regression imputation by adding the average regression variance to the regression imputations to introduce error. Stochastic regression shows much less bias than the above-mentioned techniques, but it still missed one thing – if data are imputed then intuitively one would think that more noise should be introduced to the problem than simple residual variance.[4]

#####  3.Multiple imputation
In order to deal with the problem of increased noise due to imputation, Rubin (1987)[citation needed] developed a method for averaging the outcomes across multiple imputed data sets to account for this. All multiple imputation methods follow three steps.

- Imputation – Similar to single imputation, missing values are imputed. However, the imputed values are drawn m times from a distribution rather than just once. At the end of this step, there should be m completed datasets.
- Analysis – Each of the m datasets is analyzed. At the end of this step there should be m analyses.
- Pooling – The m results are consolidated into one result by calculating the mean, variance, and confidence interval of the variable of concern.[6][7][failed verification]
Just as there are multiple methods of single imputation, there are multiple methods of multiple imputation as well. One advantage that multiple imputation has over the single imputation and complete case methods is that multiple imputation is flexible and can be used in a wide variety of scenarios. Multiple imputation can be used in cases where the data is missing completely at random, missing at random, and even when the data is missing not at random. However, the primary method of multiple imputation is multiple imputation by chained equations (MICE). It is also known as "fully conditional specification" and, "sequential regression multiple imputation." [8] MICE has been show to work very well on missing at random data, though there is evidence to suggest, through a simulation study, that with either a sufficient number of auxiliary variables it can also work on data that is missing not at random; use of a latent variable (derived through a Latent Class Analysis method produces more accurate estimates over MICE).[9]

As alluded in the previous section, single imputation does not take into account the uncertainty in the imputations. After imputation, the data is treated as if they were the actual real values in single imputation. The negligence of uncertainty in the imputation can and will lead to overly precise results and errors in any conclusions drawn.[10] By imputing multiple times, multiple imputation certainly accounts for the uncertainty and range of values that the true value could have taken.

Additionally, while it is the case that single imputation and complete case are easier to implement, multiple imputation is not very difficult to implement. There are a wide range of different statistical packages in different statistical software that readily allow someone to perform multiple imputation. For example, the MICE package allows users in R to perform multiple imputation using the MICE method.[11]



<font color='red'>Complete the following two "Missing Data" **Define, Code, and Test** sequences after watching the *"Address Missing Data First"* video.</font>

#### `treatments`: Missing records (280 instead of 350)

##### Define
*Note: the missing `treatments` records are stored in a file named `treatments_cut.csv`*

*concate the treatments dataset withe treatments_cut dataset, get a full dataset with 350 rows*

##### Code

In [142]:
treatments_cut = pd.read_csv('treatments_cut.csv')
treatments_cut.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,jožka,resanovič,22u - 30u,-,7.56,7.22,0.34
1,inunnguaq,heilmann,57u - 67u,-,7.85,7.45,NaN
2,alwin,svensson,36u - 39u,-,7.78,7.34,NaN
3,thể,lương,-,61u - 64u,7.64,7.22,0.92
4,amanda,ribeiro,36u - 44u,-,7.85,7.47,0.38


In [143]:
treatments_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 7 columns):
given_name      70 non-null object
surname         70 non-null object
auralin         70 non-null object
novodra         70 non-null object
hba1c_start     70 non-null float64
hba1c_end       70 non-null float64
hba1c_change    42 non-null float64
dtypes: float64(3), object(4)
memory usage: 4.0+ KB


In [144]:
treatments_clean.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [145]:
treatments_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
given_name      280 non-null object
surname         280 non-null object
auralin         280 non-null object
novodra         280 non-null object
hba1c_start     280 non-null float64
hba1c_end       280 non-null float64
hba1c_change    171 non-null float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [146]:
treatments_clean = pd.concat([treatments,treatments_cut],
                             ignore_index = True)


##### Test

In [147]:
treatments_clean.shape

(350, 7)

#### `treatments`: Missing HbA1c changes and Inaccurate HbA1c changes (leading 4s mistaken as 9s)
*Note: the "Inaccurate HbA1c changes (leading 4s mistaken as 9s)" observation, which is an accuracy issue and not a completeness issue, is included in this header because it is also fixed by the cleaning operation that fixes the missing "Missing HbA1c changes" observation. Multiple observations in one **Define, Code, and Test** header occurs multiple times in this notebook.*

##### Define
inplace null and wrong number in hba1c_change column with hba1c_start-hba1c_end

##### Code

In [148]:
treatments_clean.hba1c_change = treatments_clean.hba1c_start - treatments_clean.hba1c_end 


##### Test

In [149]:
treatments_clean.hba1c_change.head()

0    0.43
1    0.47
2    0.43
3    0.35
4    0.32
Name: hba1c_change, dtype: float64

##### It is better to tidy the tidiness issues (stractual issues) first, after that, data quality cleaning will be much easier.

### Tidiness

#### Contact column in `patients` table contains two variables: phone number and email

##### Define
*Hint 1: use regular expressions with pandas' [`str.extract` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html). Here is an amazing [regex tutorial](https://regexone.com/). Hint 2: [various phone number regex patterns](https://stackoverflow.com/questions/16699007/regular-expression-to-match-standard-10-digit-phone-number). Hint 3: [email address regex pattern](http://emailregex.com/), which you might need to modify to distinguish the email from the phone number.*

separate the contact column into phone_number and email using pandas' regular expressions

##### Code

In [150]:
patients_clean['phone_number'] = patients_clean.contact.str.extract(
    '(\(?\d{3}\)?[\s\-]\d{3}[\s\-]\d{4})')


In [151]:
patients_clean['email']= patients_clean.contact.str.extract(
    '([A-Z]\w*@\w+\.[A-Za-z]+)')

In [152]:
patients_clean.phone_number

0        951-719-9170
1      (217) 569-3204
2        402-363-6804
3      (732) 636-8246
4        334-515-7487
            ...      
498      207-477-0579
499      928-284-4492
500      816-223-6007
501      360 443 2060
502      402-848-4923
Name: phone_number, Length: 503, dtype: object

In [153]:
patients_clean.drop(columns = {'contact'}, inplace = True)

##### Test

In [154]:
patients_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 15 columns):
patient_id      503 non-null int64
assigned_sex    503 non-null object
given_name      503 non-null object
surname         503 non-null object
address         491 non-null object
city            491 non-null object
state           491 non-null object
zip_code        491 non-null float64
country         491 non-null object
birthdate       503 non-null object
weight          503 non-null float64
height          503 non-null int64
bmi             503 non-null float64
phone_number    485 non-null object
email           485 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 59.1+ KB


In [155]:
patients_clean.phone_number

0        951-719-9170
1      (217) 569-3204
2        402-363-6804
3      (732) 636-8246
4        334-515-7487
            ...      
498      207-477-0579
499      928-284-4492
500      816-223-6007
501      360 443 2060
502      402-848-4923
Name: phone_number, Length: 503, dtype: object

In [156]:
patients_clean.email

0               ZoeWellish@superrito.com
1                   PamelaSHill@cuvox.de
2                   JaeMDebord@gustr.com
3              PhanBaLiem@jourrapide.com
4                    TimNeudorf@cuvox.de
                     ...                
498      MustafaLindstrom@jourrapide.com
499               RumanBisliev@gustr.com
500            JinkedeKeizer@teleworm.us
501    ChidaluOnyekaozulu@jourrapide.com
502             PatrickGersten@rhyta.com
Name: email, Length: 503, dtype: object

In [157]:
patients_clean[patients_clean.email.isnull()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,birthdate,weight,height,bmi,phone_number,email
209,210,female,Lalita,Eldarkhanov,NaN,NaN,NaN,NaN,NaN,8/14/1950,143.4,62,26.2,NaN,NaN
215,216,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,1/1/1975,180.0,72,24.4,NaN,NaN
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1,NaN,NaN
229,230,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,1/1/1975,180.0,72,24.4,NaN,NaN
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4,NaN,NaN
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2,NaN,NaN
237,238,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,1/1/1975,180.0,72,24.4,NaN,NaN
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4,NaN,NaN
244,245,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,1/1/1975,180.0,72,24.4,NaN,NaN
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6,NaN,NaN


#### Three variables in two columns in `treatments` table (treatment, start dose and end dose)

##### Define
*Your definition here. Hint: use pandas' [melt function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) and [`str.split()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html). Here is an excellent [`melt` tutorial](https://deparkes.co.uk/2016/10/28/reshape-pandas-data-with-melt/).*

In [158]:
treatments_clean.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,0.43
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.47
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,0.43
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [159]:
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'],
                           var_name='treatment', value_name='dose')
treatments_clean.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u
1,elliot,richardson,7.56,7.09,0.47,auralin,-
2,yukitaka,takenaka,7.68,7.25,0.43,auralin,-
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u
4,alissa,montez,7.78,7.46,0.32,auralin,-


In [160]:
treatments_clean = treatments_clean[treatments_clean['dose'] != '-']

In [161]:
treatments_clean.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,dose
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u - 48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u - 36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u - 42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u - 38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u - 36u


In [162]:
treatments_clean[['dose_start','dose_end']] = treatments_clean.dose.str.split(pat = "-",expand = True)


In [163]:
treatments_clean.drop(columns = {'dose'},inplace =True)

In [164]:
treatments_clean.head()

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,dose_start,dose_end
0,veronika,jindrová,7.63,7.20,0.43,auralin,41u,48u
3,skye,gormanston,7.97,7.62,0.35,auralin,33u,36u
6,sophia,haugen,7.65,7.27,0.38,auralin,37u,42u
7,eddie,archer,7.89,7.55,0.34,auralin,31u,38u
9,asia,woźniak,7.76,7.37,0.39,auralin,30u,36u
